In [5]:
%%bash
# this assumes that you have a proper version of PyTorch already installed
pip install -q torchaudio omegaconf soundfile

In [6]:
%%bash
sudo apt-get install portaudio19-dev



Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (182 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [7]:
%%bash
pip uninstall sounddevice



In [8]:
%%bash
pip install sounddevice

In [9]:
import sounddevice as sd
sd.query_devices()

REAL TIME AUDIO


In [ ]:
import torch
import torchaudio
import sounddevice as sd

device = torch.device('cpu')  # gpu also works, but our models are fast enough for CPU

model, decoder, utils = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                       model='silero_stt',
                                       language='en', # also available 'de', 'es'
                                       device=device)
(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils  # see function signature for details

# Define a callback function to process audio chunks
def callback(indata, frames, time, status):
    if status:
        print(status)
    # Convert audio data to tensor
    audio_tensor = torch.tensor(indata.flatten(), dtype=torch.float32)
    audio_tensor = audio_tensor.unsqueeze(0)
    audio_tensor = audio_tensor.unsqueeze(0)

    input = prepare_model_input(audio_tensor, device=device)

    # Process audio with the model
    output = model(input)
    for example in output:
        print(decoder(example.cpu()))


AUDIO FILE

In [12]:
import torch
import torchaudio
from glob import glob

device = torch.device('cpu')  # gpu also works, but our models are fast enough for CPU

model, decoder, utils = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                       model='silero_stt',
                                       language='en', # also available 'de', 'es'
                                       device=device)
(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils  # see function signature for details

# Path to your local MP3 audio file
file_path = "/content/recording_1.mp3"

test_files = [file_path]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]),
                            device=device)

output = model(input)
for example in output:
    print(decoder(example.cpu()))


Using cache found in /root/.cache/torch/hub/snakers4_silero-models_master


please someone helpped me i think my leg is broken and i'm bleeding badly i need an ambulance right away please hurry


In [13]:
pip install transformers


In [4]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification




In [14]:
# Pick a suitable sentiment analysis model
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [24]:
def analyze_speech(text):
  """Analyzes text for panic words, sentiment, and considers urgency.

  Args:
      text: The text obtained from speech-to-text conversion.

  Returns:
      A string indicating "Critical Accident" or "Not a Critical Accident",
      along with a sentiment score (positive, negative, or neutral).
  """

  # Improved Panic Word List
  panic_words = [
      "ambulance", "pain", "bleeding", "broken", "hurt", "crash", "accident",
      "emergency", "panic", "scared", "injury", "fracture", "severe", "help"]

  # Preprocess the text
  inputs = tokenizer(text.lower(), return_tensors="pt")

  # Sentiment Analysis with Hugging Face
  output = model(**inputs)
  sentiment_label = output.logits.argmax(-1).item()
  sentiment_scores = {"positive": 0, "neutral": 0.5, "negative": 1}
  sentiment_score = sentiment_scores[sentiment]
  # Panic Word Detection and Urgency Consideration
  panic_count = sum(word in panic_words for word in text.lower().split())
  urgency_words = ["help", "emergency", "hurry"]
  urgency_count = sum(word in urgency_words for word in text.lower().split())

  # Criticality Logic with Weighted Scores (Adjust weights as needed)
  panic_weight = 0.7
  sentiment_weight = 0.3
  urgency_weight = 0.2
  criticality_score = (panic_count * panic_weight) + (sentiment_weight * sentiment_score) + (urgency_weight * urgency_count)

  if criticality_score >= 1.5:
      return "Critical Accident", sentiment
  else:
      return "Not a Critical Accident", sentiment


# Analyze the converted text
text = decoder(example.cpu())
criticality, sentiment = analyze_speech(text)
print(f"Speech Analysis: {criticality}, Sentiment: {sentiment}")




Speech Analysis: Critical Accident, Sentiment: positive


In [ ]:
import torch
import zipfile
import torchaudio
from glob import glob

device = torch.device('cpu')  # gpu also works, but our models are fast enough for CPU

model, decoder, utils = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                       model='silero_stt',
                                       language='en', # also available 'de', 'es'
                                       device=device)
(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils  # see function signature for details

# download a single file, any format compatible with TorchAudio (soundfile backend)
torch.hub.download_url_to_file('https://opus-codec.org/static/examples/samples/speech_orig.wav',
                               dst ='speech_orig.wav', progress=True)
test_files = glob('speech_orig.wav')
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]),
                            device=device)

output = model(input)
for example in output:
    print(decoder(example.cpu()))